线性回归的简洁实现
通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集

In [6]:
def synthetic_data(w, b, num_examples):
    """生成 y = Xw + b + 噪声。"""
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y = y + torch.normal(0,0.01,y.shape)
    return X ,y.reshape(-1,1)

In [7]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [11]:
def load_array(data_arrays, batch_size, is_train=True):
    "构造一个Pytorch数据迭代器"
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
batch_size = 10
data_iter = load_array((features,labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.3081,  0.0566],
         [ 2.1635,  2.0429],
         [-0.5388, -0.0290],
         [ 0.8350, -0.2062],
         [ 0.4794, -0.5743],
         [ 1.1194, -0.8138],
         [ 1.2087,  0.4139],
         [-1.0624,  1.6515],
         [ 0.7779,  1.2151],
         [-0.2799,  0.3899]]),
 tensor([[ 4.6187],
         [ 1.5773],
         [ 3.2295],
         [ 6.5922],
         [ 7.1194],
         [ 9.2184],
         [ 5.2136],
         [-3.5441],
         [ 1.6322],
         [ 2.3151]])]

使用框架的预定义好的层

In [13]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

初始化模型参数

In [15]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是MSELoss类，也称为平方L2范数

In [17]:
loss = nn.MSELoss()

实例化 SGD 实例

In [18]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [21]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000096
epoch 2, loss 0.000096
epoch 3, loss 0.000097


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [23]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 8.1038e-04, -6.6757e-06])
b的估计误差： tensor([-0.0004])
